# Environment Set Up

In [ ]:
# G drive connection
# learn: to access a shared file just put a shortcut in mi drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Libraries import

import pandas as pd
from tabulate import tabulate
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime

# Connection test

In [ ]:
# Verify reading folder

!ls '/content/drive/MyDrive/andresrokp/Thesis Juandavid/'

In [ ]:
# Hola mundo

file_path = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Companies.dta'
df = pd.read_stata(file_path)
print('\n---\ninfo\n---')
df.info()  # Show column
print('\n---\ndescribe\n---')
print(df.describe())  # Show very basic stadistics
print('\n---\nhead\n---')
df.head()  # Show first rows


# Helpers Functions

In [ ]:
def akap_pretty_print_df(df, rows):

    rows_head = rows
    rows_tail = rows

    if rows < 0:
      rows_head = -rows * 10
      rows_tail = 0

    columns = df.columns.tolist()
    ellipsis_values = ['...' for _ in columns]
    ellipsis_row = pd.DataFrame([ellipsis_values], columns=columns)
    head = df.head(rows_head)
    tail = df.tail(rows_tail)
    payload = tabulate(pd.concat([head,ellipsis_row,tail]),headers='keys', tablefmt='pretty')
    print( payload )
    return payload

dirPath = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/archivos/'

def akap_csv_file_writer(df, file_name):
    file_path = f'{dirPath}{file_name}.csv'
    df.to_csv(file_path, index=False)

def akap_txt_file_writer(df, file_name, rows):
  file_path = f'/content/drive/MyDrive/andresrokp/Thesis Juandavid/{file_name}.txt'
  columns = df.columns.tolist()
  with open(file_path,'w') as txt_file:
    txt_file.write(akap_pretty_print_df(df, rows))

# Dataset loading

In [ ]:
# File paths
path_companies_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Companies.dta'
path_ipo_m_a_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/IPO_M&A.dta'
path_investments_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investments.dta'
path_investors_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investors.dta'
path_vico = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/VICO5.0_Investors_Reputation_withOldIDs.dta'

# Read Companies data
print("Reading Companies data...")
df_companies = pd.read_stata(path_companies_file)
print("\nCompanies Data Info:")
df_companies.info()
akap_pretty_print_df(df_companies, 5)
akap_csv_file_writer(df_companies, 'df_companies')
# csv write
df_companies_clean_by_country \
  = df_companies.groupby(['CompanyNation']) \
                      .size() \
                      .reset_index(name='CompaniesNationCount') \
                      .sort_values(by='CompaniesNationCount', ascending=False) \
                      .reset_index(drop=True)
akap_csv_file_writer(df_companies_clean_by_country, 'df_companies_clean_by_country')
df_companies_clean_by_industry \
  = df_companies.groupby(['NACERev2corcodesdes']) \
                      .size() \
                      .reset_index(name='CompanyIndustryCount') \
                      .sort_values(by='CompanyIndustryCount', ascending=False) \
                      .reset_index(drop=True)
akap_csv_file_writer(df_companies_clean_by_industry, 'df_companies_clean_by_industry')

# Read IPO M&A data
print("\nReading IPO M&A data...")
df_ipo_ma = pd.read_stata(path_ipo_m_a_file)
df_ipo_ma['IPODealvaluethEUR'] = pd.to_numeric(df_ipo_ma['IPODealvaluethEUR'], errors='coerce')
print("\nIPO M&A Data Info:")
print('unique IPO_dummy: ', df_ipo_ma['IPO_dummy'].unique())
print('count IPO_dummy = 1: ', len(df_ipo_ma[df_ipo_ma['IPO_dummy'] == 1]))
print('count IPO_dummy = 0: ', len(df_ipo_ma[df_ipo_ma['IPO_dummy'] == 0]))
df_ipo_ma.info()
akap_csv_file_writer(df_ipo_ma, 'df_ipo_ma')
akap_pretty_print_df(df_ipo_ma, 5)

# Read Investors data
print("\nReading Investors data...")
df_investors = pd.read_stata(path_investors_file)
# cleaning dejar solo VCs
print("\Cleaning Investors data...")
df_investors_clean = df_investors[df_investors['InvestorType'] != 'BA']
df_investors_clean = df_investors_clean[df_investors_clean['InvestorType'] != 'Other']
print("\nInvestors Clean Data Info:")
df_investors_clean.info()
akap_pretty_print_df(df_investors_clean, 5)
akap_csv_file_writer(df_investors_clean, 'df_investors_clean')
# csv write
df_investors_clean_by_country \
  = df_investors_clean.groupby(['InvestorNation']) \
                      .size() \
                      .reset_index(name='InvestorNationCount') \
                      .sort_values(by='InvestorNationCount', ascending=False) \
                      .reset_index(drop=True)
akap_csv_file_writer(df_investors_clean_by_country, 'df_investors_clean_by_country')

# Read Investments data
print("\nReading Investments data...")
df_investments = pd.read_stata(path_investments_file)
df_investments_columns = df_investments.columns.tolist()
df_investments = pd.merge(df_investments, df_investors, on='InvestorID', how='left')
# cleaning
print("\nCleaning Investments data...")
df_investments_merge_clean = df_investments[df_investments['InvestorType'] != 'BA']
df_investments_merge_clean = df_investments_merge_clean[df_investments_merge_clean['InvestorType'] != 'Other']
df_investments_merge_clean = df_investments_merge_clean[~df_investments_merge_clean['InvestorID'].str.contains('missing')]
df_investments_clean = df_investments_merge_clean[df_investments_columns]
# output
print("\nInvestments Clean Data Info:")
print('uniques: ', len(df_investments_clean['CompanyID'].unique()))
df_investments_clean.info()
akap_pretty_print_df(df_investments_clean, 5)
akap_csv_file_writer(df_investments_clean, 'df_investments_clean')

# Read VICO5 data
# print("\nReading vico5_BNT data...")
# df_vico5_BNT = pd.read_stata(path_vico)
# print("\nVICO5 Data Info:")
# df_vico5_BNT.info()
# print("Number of Rows in vico5_BNT Data:", len(df_vico5_BNT))

# Interesting table builder

In [ ]:
'''

'''

# MERGING
print('\n\n\n----------MERGING----------')
# df_investments_clean >>InvestorID>> df_investors
df_merge_investments_investors = pd.merge(df_investments_clean, \
                                          df_investors_clean, \
                                          on='InvestorID', \
                                          how='left')
print( '\n<o>\n info df_merge_investments_investors.info() \n' )
df_merge_investments_investors.info()
akap_pretty_print_df(df_merge_investments_investors, 10)
# >> companies ON CompanyID
df_merge_investments_investors_companies = pd.merge(df_merge_investments_investors, \
                                           df_companies, \
                                           on='CompanyID', \
                                           how='left')
print( '\n<o>\n info df_merge_investments_investors_companies.info() \n' )
df_merge_investments_investors_companies.info()
akap_pretty_print_df(df_merge_investments_investors_companies, 10)
# >> aipió má
df_merge_investments_investors_companies_ipoma = pd.merge(df_merge_investments_investors_companies, \
                                                 df_ipo_ma, \
                                                 on='CompanyID', \
                                                 how='left')
print( '\n<o>\n info df_merge_investments_investors_companies_ipoma.info() \n' )
df_merge_investments_investors_companies_ipoma.info()
akap_pretty_print_df(df_merge_investments_investors_companies_ipoma, 10)



# LEAN df // STANDARIZE var NAME
print('\n\n\n----------LEAN df // STANDARIZE var NAME----------')
df_interesante_1 = df_merge_investments_investors[['CompanyID','InvestorID', 'InvestorName', 'InvestorNation', 'InvestorType','TotalEquityInvested_round_thEUR']]
print( '\n<o>\n info df_interesante_1.info() \n' )
df_interesante_1.info()
akap_txt_file_writer(df_interesante_1, 'df_merge_investments_investors', 10)

df_interesante_2 = df_merge_investments_investors_companies[['InvestmentDate','CompanyID','CompanyFUAName','NACERev2corcodesdes','FirstInvestmentYear','CompanyNation','TotalEquityInvested_round_thEUR','InvestorID','InvestorName','InvestorType','InvestorNation']]
print( '\n<o>\n info df_interesante_2.info() \n' )
df_interesante_2.info()
akap_txt_file_writer(df_interesante_2, 'df_merge_investments_investors_companies', 10)

df_interesante_3 = df_merge_investments_investors_companies_ipoma[['InvestmentDate','CompanyID','CompanyFUAName','NACERev2corcodesdes','FirstInvestmentYear','CompanyNation','TotalEquityInvested_round_thEUR','InvestorID','InvestorName','InvestorType','InvestorNation','IPO_dummy','IPODate','IPODealvaluethEUR','Acquisition_dummy','AcqDate','AcqDealvaluethEUR']]
print( '\n<o>\n info df_interesante_3.info() \n' )
df_interesante_3.info()
akap_txt_file_writer(df_interesante_3, 'df_merge_investments_investors_companies_ipoma', 10)

# readaptacion
# df_interesante = df_interesante_2

# AGRUPAMIENTOS
# print('\n\n\n----------AGRUPAMIENTOS----------')
# df_interesante_grouped = df_interesante.groupby(['InvestorID', 'InvestorName', 'InvestorNation', 'InvestorType']) \
#                                          .size() \
#                                          .reset_index(name='InvestorCount') \
#                                          .sort_values(by='InvestorCount', ascending=False) \
#                                          .reset_index(drop=True)
# print( '\n<o>\n info df_interesante_1_grouped.info() \n' )
# df_interesante_grouped.info()
# akap_pretty_print_df(df_interesante_grouped, 10)


# MOCHADA TOP 25
# print('\n\n\n----------MOCHADA TOP 25----------')
# print("\n<o>\n info df_merge_investments_investors_grouped_top25 conteo de apariciones TOP 25% \n")
# df_interesante_grouped_top25 = df_interesante_grouped.head(round(len(df_interesante_grouped)*0.25))
# akap_pretty_print_df(df_interesante_grouped_top25, 10)

In [ ]:
# df_interesante_1_grouped.groupby('InvestorNation').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
# plt.gca().spines[['top', 'right',]].set_visible(False)
# plt.show()
# print('.')
# df_interesante_1_grouped.groupby('InvestorType').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
# plt.gca().spines[['top', 'right',]].set_visible(False)
# plt.show()
# print('.')
# df_interesante_1_grouped['InvestorCount'].plot(kind='line', figsize=(8, 4), title='InvestorCount')
# plt.gca().spines[['top', 'right']].set_visible(False)
# plt.show()
# print('.')

# Modelo IPOs

In [ ]:
'''
 #   Column                           Non-Null Count  Dtype
---  ------                           --------------  -----
 0   InvestmentDate                   71077 non-null  datetime64[ns]
 1   CompanyID                        71077 non-null  object
 2   CompanyFUAName                   71077 non-null  object
 3   NACERev2corcodesdes              71077 non-null  object
 4   FirstInvestmentYear              71077 non-null  float32
 5   CompanyNation                    71077 non-null  object
 6   TotalEquityInvested_round_thEUR  53320 non-null  float64
 7   InvestorID                       71077 non-null  object
 8   InvestorName                     71077 non-null  object
 9   InvestorType                     71077 non-null  object
 10  InvestorNation                   71077 non-null  object
 11  IPO_dummy                        10064 non-null  float32
 12  IPODate                          2344 non-null   datetime64[ns]
 13  IPODealvaluethEUR                10064 non-null  object
 14  Acquisition_dummy                10064 non-null  float32
 15  AcqDate                          8278 non-null   datetime64[ns]
 16  AcqDealvaluethEUR                10064 non-null  object
 '''

df_modelo_4 = df_interesante_3

# LEAVE ONLY IPO 2014-2018
IPO_5ann = df_modelo_4[df_modelo_4['IPO_dummy'] == 1]
IPO_5ann = IPO_5ann[(IPO_5ann['IPODate'].dt.year >= 2013) & (IPO_5ann['IPODate'].dt.year <= 2018)]
print( '\n<o>\n info IPO_5ann.info() \n' )
print('uniques CompanyID: ', len(IPO_5ann['CompanyID'].unique()))
IPO_5ann.info()
akap_pretty_print_df(IPO_5ann, 20)


print('\n\n\n----------AGRUPAMIENTOS----------')

# by investor
IPO_5ann_grouped_by_investors \
  = IPO_5ann.groupby(['InvestorID', 'InvestorName']) \
                          .size() \
                          .reset_index(name='InvestorCount') \
                          .sort_values(by='InvestorCount', ascending=False) \
                          .reset_index(drop=True)
#print
print( '\n<o>\n info IPO_5ann_grouped_by_investors.info() \n' )
IPO_5ann_grouped_by_investors.info()
akap_pretty_print_df(IPO_5ann_grouped_by_investors, -5)
akap_csv_file_writer(IPO_5ann_grouped_by_investors, 'IPO_5ann_grouped_by_investors')
# plot
IPO_5ann_grouped_by_investors['InvestorCount'].plot(kind='line', figsize=(8, 4), title='InvestorCount')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.savefig('IPO_5ann_grouped_by_investors.png')
plt.show()

# by company
IPO_5ann_grouped_by_company \
  = IPO_5ann.groupby(['CompanyID']) \
                          .size() \
                          .reset_index(name='CompanyCount') \
                          .sort_values(by='CompanyCount', ascending=False) \
                          .reset_index(drop=True)
# print
print( '\n<o>\n info IPO_5ann_grouped_by_company.info() \n' )
IPO_5ann_grouped_by_company.info()
akap_pretty_print_df(IPO_5ann_grouped_by_company, 10)
akap_csv_file_writer(IPO_5ann_grouped_by_company, 'IPO_5ann_grouped_by_company')
# plot
IPO_5ann_grouped_by_company['CompanyCount'].plot(kind='line', figsize=(8, 4), title='CompanyCount')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.savefig(f'{dirPath}IPO_5ann_grouped_by_company.png')
plt.show()

# investor - company
#['InvestorName', 'InvestorNation', 'InvestorType', 'CompanyFUAName', 'CompanyNation', 'NACERev2corcodesdes']
IPO_5ann_grouped_by_investors_and_company \
  = IPO_5ann.groupby(['InvestorName','CompanyID']) \
                          .size() \
                          .reset_index(name='InvestorCompanyCount') \
                          .sort_values(by='InvestorCompanyCount', ascending=False) \
                          .reset_index(drop=True)
# print
print( '\n<o>\n info IPO_5ann_grouped_by_investors_and_company.info() \n' )
IPO_5ann_grouped_by_investors_and_company.info()
akap_pretty_print_df(IPO_5ann_grouped_by_investors_and_company, 10)
akap_csv_file_writer(IPO_5ann_grouped_by_investors_and_company, 'IPO_5ann_grouped_by_investors_and_company')
# plot
IPO_5ann_grouped_by_investors_and_company['InvestorCompanyCount'].plot(kind='line', figsize=(8, 4), title='InvestorCompanyCount')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.savefig(f'{dirPath}IPO_5ann_grouped_by_investors_and_company.png')
plt.show()

IPO_5ann_grouped_by_investorsType \
  = IPO_5ann.groupby(['InvestorType']) \
                          .size() \
                          .reset_index(name='InvestorCount') \
                          .sort_values(by='InvestorCount', ascending=False) \
                          .reset_index(drop=True)
print( '\n<o>\n info IPO_5ann_grouped_by_investorsType.info() \n' )
IPO_5ann_grouped_by_investorsType.info()
akap_pretty_print_df(IPO_5ann_grouped_by_investorsType, -1)

# get real numbers for IPODealvaluethEUR
IPO_5ann_DealEUR_sum_of_investors \
  = pd.merge(IPO_5ann_grouped_by_investors_and_company, \
              df_ipo_ma[['CompanyID','IPODealvaluethEUR']], \
              on='CompanyID', \
              how='left')
IPO_5ann_DealEUR_sum_of_investors \
  = IPO_5ann_DealEUR_sum_of_investors.groupby('InvestorName').agg({
                                                      'IPODealvaluethEUR': 'sum'
                                                      #'IPO_dummy': 'sum'
                                                  }) \
                                                  .reset_index() \
                                                  .sort_values(by='IPODealvaluethEUR', ascending=False) \
                                                  .reset_index(drop=True)
# print
print( '\n<o>\n info IPO_5ann_DealEUR_sum_of_investors.info() \n' )
IPO_5ann_DealEUR_sum_of_investors.info()
akap_pretty_print_df(IPO_5ann_DealEUR_sum_of_investors, -6)
akap_csv_file_writer(IPO_5ann_DealEUR_sum_of_investors, 'IPO_5ann_DealEUR_sum_of_investors')
print( '\n<o>\n info $ toal_investments \n' )
total_investments = IPO_5ann_DealEUR_sum_of_investors['IPODealvaluethEUR'].sum();
print(total_investments)
# plot
IPO_5ann_DealEUR_sum_of_investors['IPODealvaluethEUR'].plot(kind='line', figsize=(8, 4), title='IPODealvaluethEUR')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.savefig(f'{dirPath}IPO_5ann_DealEUR_sum_of_investors.png')
plt.show()


print( '\n<o>\n info IPO_5ann_DealEUR_sum_of_investors_NORMALIZED.info() \n' )
IPO_5ann_DealEUR_sum_of_investors['IPODealvaluethEUR_norm'] = IPO_5ann_DealEUR_sum_of_investors['IPODealvaluethEUR'] / total_investments
IPO_5ann_DealEUR_sum_of_investors.info()
akap_pretty_print_df(IPO_5ann_DealEUR_sum_of_investors, -6)
akap_csv_file_writer(IPO_5ann_DealEUR_sum_of_investors, 'IPO_5ann_DealEUR_sum_of_investors_NORMALIZED')
# plot
IPO_5ann_DealEUR_sum_of_investors['IPODealvaluethEUR_norm'].plot(kind='line', figsize=(8, 4), title='IPODealvaluethEUR NORM')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.savefig(f'{dirPath}IPO_5ann_DealEUR_sum_of_investors_NORMALIZED.png')
plt.show()



# Modelo Cuenta simple

In [ ]:
df_modelo_4 = df_interesante_3

#5chicos
# df_modelo_4 = df_modelo_4[(df_modelo_4['InvestmentDate'].dt.year >= 2013) & (df_modelo_4['InvestmentDate'].dt.year <= 2018)]

simple_investor_count = df_modelo_4['InvestorName'].value_counts().reset_index()
simple_investor_count.columns = ['InvestorName', 'InvestorCount']
# output
akap_pretty_print_df(simple_investor_count, 50)
akap_csv_file_writer(simple_investor_count, 'simple_investor_count')


simple_investor_count['InvestorCount'].describe()



# Modelo Age

In [ ]:
df_modelo_age = df_interesante_3

# df_modelo_age = df_modelo_age[(df_modelo_age['InvestmentDate'].dt.year >= 2013) & (df_modelo_age['InvestmentDate'].dt.year <= 2018)]


# Assuming 'InvestmentDate' is the correct column name representing the date of each investment
simple_investor_age = df_modelo_age.groupby('InvestorName')['InvestmentDate'].min().reset_index()
simple_investor_age.columns = ['InvestorName', 'EarliestInvestmentDate']
# Calculate the duration between the earliest investment date and 2018
reference_date = pd.to_datetime('2018-12-31')
simple_investor_age['YearsSinceFirstInvestment'] = (
    (reference_date - simple_investor_age['EarliestInvestmentDate']).dt.days / 365.25
)
simple_investor_age = simple_investor_age.sort_values(by="YearsSinceFirstInvestment", ascending=False)
# output
akap_pretty_print_df(simple_investor_age, 50)
akap_csv_file_writer(simple_investor_age, 'simple_investor_age')

simple_investor_age['YearsSinceFirstInvestment'].describe()

# Modelo Syndication

In [ ]:
df_modelo_syndication = df_interesante_3
df_modelo_syndication = df_modelo_syndication[(df_modelo_syndication['InvestmentDate'].dt.year >= 2013) & (df_modelo_syndication['InvestmentDate'].dt.year <= 2018)]

print( '\n<o>\n info df_modelo_syndication.info() \n' )
df_modelo_syndication.info()
print('"Bpi Groupe" count : ', df_modelo_syndication['InvestorName'].value_counts().get("Bpi Groupe", 0))
print('"Elevencooperatief Ua" count : ', df_modelo_syndication['InvestorName'].value_counts().get("Elevencooperatief Ua", 0))
print('"Elaia Partners" count : ', df_modelo_syndication['InvestorName'].value_counts().get("Elaia Partners", 0))
akap_pretty_print_df(df_modelo_syndication,-2)

# Step 1: parejas investor-company únicas
unique_pairs_inv_comp = df_modelo_syndication[['InvestorName', 'CompanyID']].drop_duplicates()
print('"Bpi Groupe" count : ', unique_pairs_inv_comp['InvestorName'].value_counts().get("Bpi Groupe", 0))
print('"Elevencooperatief Ua" count : ', unique_pairs_inv_comp['InvestorName'].value_counts().get("Elevencooperatief Ua", 0))
print('"Elaia Partners" count : ', unique_pairs_inv_comp['InvestorName'].value_counts().get("Elaia Partners", 0))
print( '\n<o>\n info unique_pairs_inv_comp.info() \n' )
unique_pairs_inv_comp.info()
akap_pretty_print_df(unique_pairs_inv_comp,-2)

# agrupar por compañia y número de inversores
grouped_by_company = unique_pairs_inv_comp.groupby('CompanyID')['InvestorName'] \
                     .size() \
                     .reset_index(name='InvestorsCountInCompany') \
                     .sort_values(by='InvestorsCountInCompany', ascending=False) \
                     .reset_index(drop=True)
# quedarse solo con los que tengan a más de 1
grouped_by_company = grouped_by_company[grouped_by_company['InvestorsCountInCompany'] > 1]
print( '\n<o>\n info grouped_by_company.info() \n' )
grouped_by_company.info()
akap_pretty_print_df(grouped_by_company,20)

# dejar solo los pairs cuya company genera syndication
only_pairs_of_syndicalizing_companies = unique_pairs_inv_comp.merge(grouped_by_company, on='CompanyID', how='inner')[['InvestorName','CompanyID']]
print( '\n<o>\n info only_pairs_of_syndicalizing_companies.info() \n' )
only_pairs_of_syndicalizing_companies.info()
akap_pretty_print_df(only_pairs_of_syndicalizing_companies,20)

# contar cuantas veces cada Investor tiene Company que generó syndication
investors_syndication_count = unique_pairs_inv_comp.groupby('InvestorName')['InvestorName'] \
                                .size() \
                                .reset_index(name='InvestorsSyndicationCount') \
                                .sort_values(by='InvestorsSyndicationCount', ascending=False) \
                                .reset_index(drop=True)
print( '\n<o>\n info investors_syndication_count.info() \n' )
investors_syndication_count.info()
akap_pretty_print_df(investors_syndication_count,20)

akap_csv_file_writer(investors_syndication_count,'investors_syndication_count_5chicorios_noIPO')


# Otros

In [ ]:
# Check runtime vars
# dir() keeps trak of the declared variables in the runtime

[var for var in dir() if '_' != var[0]]

In [ ]:
investor_count = df_investments['InvestorID'].value_counts().get('VCInvestor002335', 0)
investor_count